In [3]:
import os 
import json
import pprint
%matplotlib inline
from serializer import*
import tweepy
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
from pyvis.network import Network
import os

#Twitter API credentials
api_key = "ENFvcEFBU6prbQWZVMKYHhHCW"
api_secret = "OoLxYiucfTgnheUH1Bp7YJTsyIto9VcxLERBeXCg84NwjjgCAr"
access_token = "2572511777-adhDu1rnH5RYVPdfk51jPzLbOzpppivKmVYTtFH"
access_secret = "Oq1Q8CTYoDKxfyal7aY2x5V88QBqM2SAFac5e8LV6eeWY"
bearer_token = "AAAAAAAAAAAAAAAAAAAAAJkTJQEAAAAAaoRAjF4yb5OBca0iPYQZvs5vH5s%3D5AG5ZaEGM6pFtGL7cQohdtzimqd1aLcqHGbJPOSCXO4Fxnltwz"

In [2]:
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
if(api.verify_credentials):
    print ('Authentication completed successfully!')
    
s = Serializer("data")

Authentication completed successfully!


In [ ]:
users = ["mizzaro"]

for user in users:
    print(f"Processing @{user}")

    user_followers = []

    for item in tweepy.Cursor(
            api.friends,
            screen_name=user,
            skip_status=True,
            include_user_entities=False
    ).items():
        user_followers.append(item._json)

    print(f"Found {len(user_followers)} following for @{user}")
    s.serialize_json(f"{user}/{user}_following.json", user_followers)

Processing @mizzaro


Rate limit reached. Sleeping for: 892


In [12]:
import os

directory = "data"
G = nx.DiGraph(membri=["Lorenzo Bellina" "Francesco Bombassei De Bona", "Andrea Cantarutti", "Gabriele Dominici"])
for dirname in os.listdir(directory):
    if not dirname.startswith(".") and not dirname == "all_users.json":
        print(f"Analizzo {dirname}")
        s = Serializer(f"data/{dirname}")
        profile = s.read_json(f"{dirname}_profile.json")
        G.add_node(profile["id"], **profile)
        print(f'Aggiungo il nodo {profile["name"]}')

        followers = s.read_json(f"{dirname}_followers.json")
        print(f'Aggiungo i {len(followers)} followers di {profile["name"]}')
        for item in followers:
            G.add_node(item["id"], **item)
            G.add_edge(item["id"], profile["id"])

        following = s.read_json(f"{dirname}_following.json")
        print(f'Aggiungo i {len(following)} following di {profile["name"]}')
        for item in following:
            G.add_node(item["id"], **item)
            G.add_edge(profile["id"], item["id"])

        print(f'Inserisco i random di {profile["name"]}')
        temp_dir = directory+"/"+dirname
        for file in os.listdir(temp_dir):
            if file.startswith("random"):
                id_user = int(file.split("_")[1])
                if file.endswith("following.json"):
                    following = s.read_json(f"{file}")
                    print(f"Inserisco i {len(following)} following del random di id {id_user}")
                    for item in following:
                        G.add_node(item["id"], **item)
                        G.add_edge(id_user, item["id"])
                elif file.endswith("follower.json"):
                    followers = s.read_json(f"{file}")
                    print(f"Inserisco i {len(followers)} followers del random di id {id_user}")
                    for item in followers:
                        G.add_node(item["id"], **item)
                        G.add_edge(item["id"], id_user)
        node_it = G.in_degree()
        for (node, in_degree) in node_it:
            G.nodes[node]["follower_finded"] = in_degree 
            
nx.write_gpickle(G, "graph/graph_networkx.pkl")

Analizzo damiano10
Data read from path: data/damiano10/damiano10_profile.json
Aggiungo il nodo Damiano Spina
Data read from path: data/damiano10/damiano10_followers.json
Aggiungo i 786 followers di Damiano Spina
Data read from path: data/damiano10/damiano10_following.json
Aggiungo i 836 following di Damiano Spina
Inserisco i random di Damiano Spina
Data read from path: data/damiano10/random_1019221160742014976_follower.json
Inserisco i 10 followers del random di id 1019221160742014976
Data read from path: data/damiano10/random_1037548569467731968_following.json
Inserisco i 10 following del random di id 1037548569467731968
Data read from path: data/damiano10/random_1217315090_following.json
Inserisco i 10 following del random di id 1217315090
Data read from path: data/damiano10/random_135508015_follower.json
Inserisco i 10 followers del random di id 135508015
Data read from path: data/damiano10/random_201194687_follower.json
Inserisco i 10 followers del random di id 201194687
Data read 

In [4]:
def get_friendship(sourceid, targetid, api):
    kind = ""

    friendship = api.show_friendship(source_id=sourceid, target_id=targetid)

    if not friendship[0].following and not friendship[0].followed_by:
        kind = "none"
    elif not friendship[0].following and friendship[0].followed_by:
        kind = "r_l"
    elif friendship[0].following and not friendship[0].followed_by:
        kind = "l_r"
    else:
        kind = "bi"

    return {
        "source_id": sourceid,
        "target_id": targetid,
        "friendship": kind
    }

accounts = ["damiano10"]
serializer = Serializer("data")
users = serializer.read_json("all_users.json")
edges = []
for account in accounts:
    serializer = Serializer(f'data/{account}')
    account_json = serializer.read_json(f"{account}_profile.json")
    account_id = account_json["id"]
    for user in users:
        if user["id"] is not account_id:
            edges.append(get_friendship(account_id, user["id"], api))
            print(f"Added friendship between {account} and {user['screen_name']}")
    serializer.serialize_json(f'{account}_friendships.json', edges)

Data read from path: data/all_users.json
Data read from path: data/damiano10/damiano10_profile.json
Added friendship between damiano10 and ChriShot90
Added friendship between damiano10 and dungdx34
Added friendship between damiano10 and flaviomartins
Added friendship between damiano10 and WWRob
Added friendship between damiano10 and JCAlbornozC
Added friendship between damiano10 and FTHopf
Added friendship between damiano10 and lgr3d
Added friendship between damiano10 and antonio_mallia
Added friendship between damiano10 and NewsIR_Workshop
Added friendship between damiano10 and alb_purpura
Added friendship between damiano10 and sculpepper
Added friendship between damiano10 and ntcirstc
Added friendship between damiano10 and acm_ictir
Added friendship between damiano10 and emranrian55
Added friendship between damiano10 and ida_mele
Added friendship between damiano10 and EvALL_TEAM
Added friendship between damiano10 and heartedlisa30
Added friendship between damiano10 and ABH878
Added f

Added friendship between damiano10 and makkox
Added friendship between damiano10 and ecir2020
Added friendship between damiano10 and sigirf
Added friendship between damiano10 and firadl
Added friendship between damiano10 and jallan_umass
Added friendship between damiano10 and LiveSpinoza
Added friendship between damiano10 and SciVal
Added friendship between damiano10 and ircdl2019
Added friendship between damiano10 and sigir2018
Added friendship between damiano10 and SpotifyResearch
Added friendship between damiano10 and learnIRWSDM
Added friendship between damiano10 and varphi
Added friendship between damiano10 and device_null
Added friendship between damiano10 and cyby
Added friendship between damiano10 and climagic
Added friendship between damiano10 and SpaceX


Rate limit reached. Sleeping for: 178


Added friendship between damiano10 and pnbennett
Added friendship between damiano10 and AndrewYNg
Added friendship between damiano10 and RivalRec
Added friendship between damiano10 and DavidELosada
Added friendship between damiano10 and srchvrs
Added friendship between damiano10 and ecir2018
Added friendship between damiano10 and ntonellotto
Added friendship between damiano10 and anjiefang
Added friendship between damiano10 and WSDMSocial
Added friendship between damiano10 and ladamic
Added friendship between damiano10 and nick_craswell
Added friendship between damiano10 and UnderdogGeek
Added friendship between damiano10 and bemikelive
Added friendship between damiano10 and jure
Added friendship between damiano10 and maxIRIT
Added friendship between damiano10 and davidcarmel
Added friendship between damiano10 and DESIRES_IR
Added friendship between damiano10 and ipeirotis
Added friendship between damiano10 and stefhaustein
Added friendship between damiano10 and ICTIR17
Added friendshi

Added friendship between damiano10 and HCIR_GeneG
Added friendship between damiano10 and tetsuyasakai
Added friendship between damiano10 and metzlerd
Added friendship between damiano10 and serdyukovp
Added friendship between damiano10 and YahooResearch
Added friendship between damiano10 and lintool
Added friendship between damiano10 and ian_soboroff
Added friendship between damiano10 and elunca
Added friendship between damiano10 and ericschmidt
Added friendship between damiano10 and MSFTResearch
Added friendship between damiano10 and theamitsinghal
Added friendship between damiano10 and peter_r_bailey
Added friendship between damiano10 and stanfordnlp
Added friendship between damiano10 and GoogleAI


...

Rate limit reached. Sleeping for: 838


Added friendship between damiano10 and bh46824
Added friendship between damiano10 and AnastasiaKavada
Added friendship between damiano10 and BethSager5
Added friendship between damiano10 and DataChildFuture
Added friendship between damiano10 and XuanWei59983889
Added friendship between damiano10 and TechSochSurrey
Added friendship between damiano10 and LuncaW
Added friendship between damiano10 and HUJiaying35
Added friendship between damiano10 and ana__manzano
Added friendship between damiano10 and ClareChilds3
Added friendship between damiano10 and Laura_36726
Added friendship between damiano10 and Lauinlondon
Added friendship between damiano10 and tiagoh
Added friendship between damiano10 and GoodlyLabs
Added friendship between damiano10 and GiuliP93
Added friendship between damiano10 and lior_gazit
Added friendship between damiano10 and AliFakeri
Added friendship between damiano10 and JoannaRoperFCDO
Added friendship between damiano10 and vanpelt
Added friendship between damiano10 a

Added friendship between damiano10 and smithsonian
Added friendship between damiano10 and verge
Added friendship between damiano10 and Flipboard
Added friendship between damiano10 and FVGlive
Added friendship between damiano10 and ComuneDiUdine
Added friendship between damiano10 and GoogleDoodles
Added friendship between damiano10 and guardian
Added friendship between damiano10 and Food52
Added friendship between damiano10 and smittenkitchen
Added friendship between damiano10 and dariabig
Added friendship between damiano10 and HuffPost
Added friendship between damiano10 and ardissfvg
Added friendship between damiano10 and wireditalia
Added friendship between damiano10 and california_bee
Added friendship between damiano10 and TheSimpsons
Added friendship between damiano10 and NatGeoPhotos
Added friendship between damiano10 and cnni
Added friendship between damiano10 and NASA


Rate limit reached. Sleeping for: 845


Added friendship between damiano10 and TheAcademy
Added friendship between damiano10 and BBCWorld
Added friendship between damiano10 and BeingASir
Added friendship between damiano10 and TIME
Added friendship between damiano10 and ilpost
Added friendship between damiano10 and GameOfThrones
Added friendship between damiano10 and jk_rowling
Added friendship between damiano10 and AstroSamantha
Added friendship between damiano10 and EmmaWatson
Added friendship between damiano10 and Internazionale
Added friendship between damiano10 and SkyTG24
Added friendship between damiano10 and dbpedia
Added friendship between damiano10 and Genesys
Added friendship between damiano10 and MaastrichtU
Added friendship between damiano10 and us_bridge
Added friendship between damiano10 and vinaysetty
Added friendship between damiano10 and TIB_SDM
Added friendship between damiano10 and stefaniafanaru
Added friendship between damiano10 and Yi_Benevolence
Added friendship between damiano10 and EAmiraghyan
Added 

In [10]:
directory = "data"
G = nx.Graph(membri=["Lorenzo Bellina" "Francesco Bombassei De Bona", "Andrea Cantarutti", "Gabriele Dominici"])
s = Serializer(f"{directory}")
all_nodes = s.read_json(f"all_users.json")
print(len(list(all_nodes)))
for profile in all_nodes:
    G.add_node(profile["id"], **profile)
    print(f'Aggiungo il nodo {profile["name"]}')
all_edges = s.read_json(f"all_friendships.json")
for friendship in all_edges:
    if(friendship["friendship"] != "none"):
        G.add_edge(friendship["source_id"], friendship["target_id"], value = friendship["friendship"])
        print(f'Aggiungo l arco tra {friendship["source_id"]} e {friendship["target_id"]}')

print(len(list(G.nodes)))
print(len(list(G.edges)))
            
nx.write_gpickle(G, "graph/graph_networkx.pkl")

Data read from path: data/all_users.json
2605
Aggiungo il nodo Christian Abbondo
Aggiungo il nodo Dung Doan
Aggiungo il nodo Flavio Martins
Aggiungo il nodo Robert Monarch
Aggiungo il nodo Jorge Carrillo-de-A.
Aggiungo il nodo Frank T. Hopfgartner
Aggiungo il nodo Luke Gallagher
Aggiungo il nodo Antonio Mallia
Aggiungo il nodo NewsIR Workshop '19
Aggiungo il nodo Alberto Purpura
Aggiungo il nodo J. Shane Culpepper
Aggiungo il nodo NTCIR STC
Aggiungo il nodo ACM SIGIR ICTIR
Aggiungo il nodo Emran Hossain
Aggiungo il nodo Ida Mele
Aggiungo il nodo EvALL
Aggiungo il nodo Lisa Smith
Aggiungo il nodo Ali Bahrainian
Aggiungo il nodo essir2017
Aggiungo il nodo Franco Maria Nardini
Aggiungo il nodo Md Zia Ullah
Aggiungo il nodo Josiane Mothe
Aggiungo il nodo RankSys
Aggiungo il nodo ACM SIGIR
Aggiungo il nodo Adrian CHIFU
Aggiungo il nodo Eric Gaussier
Aggiungo il nodo AIRS 2016
Aggiungo il nodo Kevin Roitero
Aggiungo il nodo AB 🍪 ENJOY MY GAMES 🌏
Aggiungo il nodo Dyaa Albakour
Aggiungo il nod

Aggiungo il nodo javier
Aggiungo il nodo Monica Noselli
Aggiungo il nodo jenniffer magruder
Aggiungo il nodo Jessica Kegu
Aggiungo il nodo Timnit Gebru
Aggiungo il nodo CIKM 2021
Aggiungo il nodo Carlo (just vote!) Curino
Aggiungo il nodo Christos Christodoulopoulos
Aggiungo il nodo NaomiOsaka大坂なおみ
Aggiungo il nodo Lugano
Aggiungo il nodo USI University
Aggiungo il nodo HumanitiesUQ
Aggiungo il nodo Professor Katharine Gelber, PhD
Aggiungo il nodo Sarah Percy
Aggiungo il nodo bruce hawker
Aggiungo il nodo Jenny Thompson
Aggiungo il nodo Michael Rowland
Aggiungo il nodo Leigh Sales
Aggiungo il nodo Adji Bousso Dieng
Aggiungo il nodo First Draft
Aggiungo il nodo Claire Wardle
Aggiungo il nodo Julie Vercelloni
Aggiungo il nodo Axios
Aggiungo il nodo Matt Oczkowski
Aggiungo il nodo Joe Biden
Aggiungo il nodo Oversight Board
Aggiungo il nodo Paul S-H (A1674)
Aggiungo il nodo AKBC 2020
Aggiungo il nodo Rosalie Gillett
Aggiungo il nodo NYU Abu Dhabi
Aggiungo il nodo Wiki Workshop 2020
Aggiung

Aggiungo l arco tra 132646210 e 872544691442466816
Aggiungo l arco tra 132646210 e 891193292385288193
Aggiungo l arco tra 132646210 e 3310895591
Aggiungo l arco tra 132646210 e 887627169765130241
Aggiungo l arco tra 132646210 e 1598324696
Aggiungo l arco tra 132646210 e 880523990883848192
Aggiungo l arco tra 132646210 e 3406278903
Aggiungo l arco tra 132646210 e 2289023460
Aggiungo l arco tra 132646210 e 877823703249068032
Aggiungo l arco tra 132646210 e 873162092819488769
Aggiungo l arco tra 132646210 e 78269397
Aggiungo l arco tra 132646210 e 2613410444
Aggiungo l arco tra 132646210 e 838930153665363968
Aggiungo l arco tra 132646210 e 151490538
Aggiungo l arco tra 132646210 e 837214523547353089
Aggiungo l arco tra 132646210 e 856509174473248768
Aggiungo l arco tra 132646210 e 709476815090413568
Aggiungo l arco tra 132646210 e 3102522680
Aggiungo l arco tra 132646210 e 1557796237
Aggiungo l arco tra 132646210 e 327066310
Aggiungo l arco tra 132646210 e 850170113504325632
Aggiungo l ar

Aggiungo l arco tra 3036907250 e 608052226
Aggiungo l arco tra 3036907250 e 889657085868298240
Aggiungo l arco tra 3036907250 e 3102522680
Aggiungo l arco tra 3036907250 e 26240101
Aggiungo l arco tra 3036907250 e 824812182114164736
Aggiungo l arco tra 3036907250 e 39697728
Aggiungo l arco tra 3036907250 e 391867472
Aggiungo l arco tra 3036907250 e 18932422
Aggiungo l arco tra 3036907250 e 1195040516318093312
Aggiungo l arco tra 3036907250 e 762093631
Aggiungo l arco tra 3036907250 e 87476034
Aggiungo l arco tra 3036907250 e 1111315973125169152
Aggiungo l arco tra 3036907250 e 1039052508193284096
Aggiungo l arco tra 3036907250 e 720389270335135745
Aggiungo l arco tra 3036907250 e 928760788306878464
Aggiungo l arco tra 3036907250 e 966411199649808386
Aggiungo l arco tra 3036907250 e 4027105402
Aggiungo l arco tra 3036907250 e 48008938
Aggiungo l arco tra 3036907250 e 109598348
Aggiungo l arco tra 3036907250 e 783214
Aggiungo l arco tra 3036907250 e 1734575215
Aggiungo l arco tra 3036907